In [175]:
import pandas as pd
from nltk.corpus import stopwords
import re
import warnings
warnings.filterwarnings("ignore")

In [176]:
#Initialise data path
data_path = "/home/jovyan/input/data/"

In [179]:
#Read train data
data = pd.read_parquet(data_path + "train.parquet")[:1000]

In [178]:
# test data checking
test_data = pd.read_parquet(data_path + "test.parquet")

In [174]:
print(test_data.code.iloc[4],"\n"*3)

def consume_messages(self, max_next_messages):
        if self.__next_messages == 0:
            self.set_next_messages(min(1000, max_next_messages))
        self.set_next_messages(min(self.__next_messages, max_next_messages))
        mark = time.time()
        for record in self._get_messages_from_consumer():
            yield record.partition, record.offset, record.key, record.value
        newmark = time.time()
        if newmark - mark > 30:
            self.set_next_messages(self.__next_messages / 2 or 1)
        elif newmark - mark < 5:
            self.set_next_messages(min(self.__next_messages + 100, max_next_messages)) 





In [173]:
test_data.docstring.iloc[4]

'Get messages batch from Kafka (list at output)'

In [137]:
sample_data = data

In [138]:
# sample_data.groupby('y_true')['id'].count().plot.bar()

In [139]:
sample_data['code_n_words']=sample_data['code'].apply(lambda row:len(row.split(" ")))
sample_data['doc_n_words']=sample_data['docstring'].apply(lambda row:len(row.split(" ")))

In [140]:
# def normalized_word_Common(row):
#         w1 = set(map(lambda word:word.lower().strip() , row['code'].split()))
#         w2 = set(map(lambda word:word.lower().strip() , row['docstring'].split()))
#         return 1.0*len(w1 & w2)
    

# def normalized_word_Total(row):
#     w1 = set(map(lambda word: word.lower().strip(), row['code'].split()))
#     w2 = set(map(lambda word: word.lower().strip(), row['docstring'].split()))    
#     return 1.0 * (len(w1) + len(w2))


# def normalized_word_share(row):
#     w1 = set(map(lambda word: word.lower().strip(), row['code'].split()))
#     w2 = set(map(lambda word: word.lower().strip(), row['docstring'].split()))    
#     return 1.0 * len(w1 & w2)/(len(w1) + len(w2))

# sample_data['word_common'] = sample_data.apply(
#     normalized_word_Common , axis=1
# )
# sample_data['word_Total'] = sample_data.apply(
#     normalized_word_Total, axis=1
# )
# sample_data['word_share'] = sample_data.apply(
#     normalized_word_share, axis=1
# )

In [141]:
# sample_data.head()

In [142]:
print ("Number of code with minimum length code :", sample_data[sample_data['code_n_words']== 1].shape[0])
print ("Number of docstring with minimum length doc :", sample_data[sample_data['doc_n_words']== 1].shape[0])

Number of code with minimum length code : 0
Number of docstring with minimum length doc : 1983


## Analysis

In [143]:
# sample_data[(sample_data['doc_n_words']== 1) & (sample_data['y_true']== 0)]['code'].iloc[5].split('\n')

In [144]:
sample_data[(sample_data['doc_n_words']== 1) & (sample_data['y_true']== 0)].head()

,id,code,docstring,y_true,code_n_words,doc_n_words
1154,d693dd78-c5a1-4465-be56-796bbf1c4a74,"def extra_context(self, request, context):\n ...",判断两条线段是否交叉,0,25,1
1657,4c5cd695-1b95-481d-8436-7c30ff0df298,"def to_keypoint_image(self, size=1):\n ...",给出交易具体时间,0,267,1
1966,c2a6ce73-b58a-42ba-91ff-a114000124cf,def opensearch(request):\n contact_email = ...,stub,0,85,1
5351,0df5f855-eaa9-4b55-b1f8-cc110a6df8dd,"def list_space_systems(self, page_size=None):\...",https://picamera.readthedocs.io/en/release-1.1...,0,128,1
5962,8f375898-d95b-4b36-a340-5d4c13d8c665,def get_identifiers_splitted_by_weights(identi...,Lower.,0,466,1


In [145]:
def remove_non_english_words(text):
    new_string=re.sub('[^a-zA-Z0-9]',' ',text)
    # print()
    return new_string.strip()

In [146]:
# Adding function to clean https links
import string

def convert_html_text(text):
    """ function convert http link to text"""
    
    if 'https://' in text:
        for pattern in string.punctuation:
            text = text.replace(pattern," ")

        return text
    else:
        return text

In [147]:
# conveting hyperlinks to string format
sample_data['docstring'] = sample_data['docstring'].apply(
    convert_html_text
)

In [148]:
# !pip install -q distance

In [149]:
def clean_data(lines):
    cleaned = []
    for line in lines.split("\n"):
        line = re.sub(r"([0-9]+)","", line)
        line = re.sub(r"([0-9]+)","", line)
    
        clean = re.sub(r"""
               [,.;@#?!&$()|<~='^\\_`:>"%/{}*]+  # Accept one or more copies of punctuation
               \ *           # plus zero or more copies of a space,
               """,
               " ",          # and replace it with a single space
               line, flags=re.VERBOSE)
        #Manually handle cases not accepted by sub
        clean = clean.replace("[", "")
        clean = clean.replace("+", "")
        clean = clean.replace("]", "")
        clean = clean.replace("-", "")
        # tokenize on white space
        line = clean.split()
        # convert to lower case
        line = [word.lower() for word in line]
        # store as string
        cleaned.append(' '.join(line))
    # remove empty strings
    return " ".join(cleaned)

In [150]:
import distance

In [1]:
SAFE_DIV = 0.0001

STOP_WORDS = stopwords.words("english")

def get_token_features(q1, q2):
    token_features = [0.0]*10
    
    # Converting the Sentence into Tokens: 
    q1_tokens = q1.split()
    q2_tokens = q2.split()

    if len(q1_tokens) == 0 or len(q2_tokens) == 0:
        return token_features
    # Get the non-stopwords in Questions
    q1_words = set([word for word in q1_tokens if word not in STOP_WORDS])
    q2_words = set([word for word in q2_tokens if word not in STOP_WORDS])
    
    #Get the stopwords in Questions
    q1_stops = set([word for word in q1_tokens if word in STOP_WORDS])
    q2_stops = set([word for word in q2_tokens if word in STOP_WORDS])
    
    # Get the common non-stopwords from Question pair
    common_word_count = len(q1_words.intersection(q2_words))
    
    # Get the common stopwords from Question pair
    common_stop_count = len(q1_stops.intersection(q2_stops))
    
    # Get the common Tokens from Question pair
    common_token_count = len(set(q1_tokens).intersection(set(q2_tokens)))

    token_features[0] = common_word_count / (min(len(q1_words), len(q2_words)) + SAFE_DIV)
    token_features[1] = common_word_count / (max(len(q1_words), len(q2_words)) + SAFE_DIV)
    token_features[2] = common_stop_count / (min(len(q1_stops), len(q2_stops)) + SAFE_DIV)
    token_features[3] = common_stop_count / (max(len(q1_stops), len(q2_stops)) + SAFE_DIV)
    token_features[4] = common_token_count / (min(len(q1_tokens), len(q2_tokens)) + SAFE_DIV)
    token_features[5] = common_token_count / (max(len(q1_tokens), len(q2_tokens)) + SAFE_DIV)
    
    # Last word of both question is same or not
    token_features[6] = int(q1_tokens[-1] == q2_tokens[-1])
    
    # First word of both question is same or not
    token_features[7] = int(q1_tokens[0] == q2_tokens[0])
    
    token_features[8] = abs(len(q1_tokens) - len(q2_tokens))
    
    #Average Token Length of both Questions
    token_features[9] = (len(q1_tokens) + len(q2_tokens))/2
    return token_features

# get the Longest Common sub string

def get_longest_substr_ratio(a, b):
    strs = list(distance.lcsubstrings(a, b))
    if len(strs) == 0:
        return 0
    else:
        return len(strs[0]) / (min(len(a), len(b)) + 1)

def extract_features(df):
    # preprocessing each question
    df["code"] = df["code"].fillna("").apply(clean_data)
    df["docstring"] = df["docstring"].fillna("").apply(clean_data)

    print("token features...")
    
    # Merging Features with dataset
    token_features = df.apply(lambda x: get_token_features(x["code"], x["docstring"]), axis=1)
    
    df["cwc_min"]       = list(map(lambda x: x[0], token_features))
    df["cwc_max"]       = list(map(lambda x: x[1], token_features))
    df["csc_min"]       = list(map(lambda x: x[2], token_features))
    df["csc_max"]       = list(map(lambda x: x[3], token_features))
    df["ctc_min"]       = list(map(lambda x: x[4], token_features))
    df["ctc_max"]       = list(map(lambda x: x[5], token_features))
    df["last_word_eq"]  = list(map(lambda x: x[6], token_features))
    df["first_word_eq"] = list(map(lambda x: x[7], token_features))
    df["abs_len_diff"]  = list(map(lambda x: x[8], token_features))
    df["mean_len"]      = list(map(lambda x: x[9], token_features))
    
    print("Feature creation is done")
    
    return df

NameError: name 'stopwords' is not defined

In [152]:
%time
sample_data = extract_features(sample_data)

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 9.3 µs
token features...
Feature creation is done


In [153]:
# sample_data.head()

In [154]:
def remove_words(text):
    """ Removing words whose length less then 3 or equal to"""
    
    text = [x for x in text.split() if len(x)>3]
    
    return " ".join(text)

In [155]:
%time

sample_data['code'] = sample_data['code'].apply(remove_words)
sample_data['docstring'] = sample_data['docstring'].apply(remove_words)

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 9.54 µs


In [156]:
# print(remove_non_english_words(
# sample_data[(sample_data['doc_n_words']== 1) & (sample_data['y_true']== 1)]['docstring'].iloc[7]
# ))

In [157]:
%time
sample_data['code_processed'] = sample_data['code'].apply(remove_non_english_words)
sample_data['docstring_processed'] = sample_data['docstring'].apply(remove_non_english_words)

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 9.3 µs


In [75]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.stem import WordNetLemmatizer
 
# Create WordNetLemmatizer object
wnl = WordNetLemmatizer()
def getLemmatize_word(text):
    
    return " ".join([wnl.lemmatize(words) for words in text.split(" ")])

[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/jovyan/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [76]:
sample_data.head(2)

,id,code,docstring,y_true,codelen,docstringlen,code_n_words,doc_n_words,word_common,word_Total,...,cwc_max,csc_min,csc_max,ctc_min,ctc_max,last_word_eq,first_word_eq,abs_len_diff,mean_len,token_set_ratio
0,66236bf5-ce26-4401-82e6-1660c718008f,def get uids self filename none self update re...,return a list of uids filename unused for api ...,1,143,77,10,12,2.0,22.0,...,0.199999,0.499975,0.249994,0.363633,0.199999,0.0,0.0,9.0,15.5,55
1,b6a4ef2c-8109-486c-a9ad-4f20d2c95fb2,def purge cdn object self container obj email ...,removes a cdnenabled object from public access...,1,145,337,8,54,0.0,54.0,...,0.159999,0.000000,0.000000,0.199999,0.072727,0.0,0.0,35.0,37.5,59


In [77]:
%time
sample_data['code'] = sample_data['code'].apply(getLemmatize_word)
sample_data['docstring'] = sample_data['docstring'].apply(getLemmatize_word)

In [93]:
sample_data.code.iloc[0]

'uids self filename none self update return abook self bookentry entry self book sections'

In [192]:
from nltk.corpus import wordnet
def synonym_antonym_extractor(phrase):
     
    synonyms = []
    antonyms = []

    for syn in wordnet.synsets(phrase):
        for l in syn.lemmas():
            synonyms.append(l.name())
            if l.antonyms():
                antonyms.append(l.antonyms()[0].name())

    print(set(synonyms))
    print(set(antonyms))

synonym_antonym_extractor('time')

{'meter', 'prison_term', 'clip', 'clock_time', 'time', 'clock', 'fourth_dimension', 'metre', 'sentence'}
set()


In [78]:
# importing modules
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

ps = PorterStemmer() #creating an instance of the class

def get_stemming_word(text):
    
    return " ".join([ps.stem(words) for words in text.split(" ")])


In [104]:
## Using word_tokenize
!pip install -q tokenizers

In [79]:
sample_data['code'] = sample_data['code'].apply(get_stemming_word)
sample_data['docstring'] = sample_data['docstring'].apply(get_stemming_word)

In [158]:
sample_data.to_csv("processed_data2.csv",index=False)

In [ ]:
# Creating some features

## Method name feature

In [186]:
data.code.iloc[0].split(":")[0].split("def")[-1].strip().split("(")[0]

'get_uids'

In [187]:
def get_method_name(code:str)->str:
    """Extracting method name from method name as a feature"""
    
    return code.split(":")[0].split("def")[-1].strip().split("(")[0]